In [ ]:
# %pip install llama-index llama-index-vector-stores-upstash upstash-vector

In [1]:
import setup
setup.init_django()

In [2]:
django_pdf = "data/django.pdf"

In [13]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.upstash import UpstashVectorStore # changed from the LlamaIndex Docs

In [4]:
reader = SimpleDirectoryReader(input_files=[django_pdf])

In [6]:
import helpers
import openai
openai.api_key = helpers.config("OPENAI_API_KEY", default=None)

In [8]:
url=helpers.config("UPSTASH_VECTOR_URL", default=None)
token=helpers.config("UPSTASH_VECTOR_TOKEN", default=None)
url

'https://grown-basilisk-88477-us1-vector.upstash.io'

In [5]:
data = reader.load_data()

In [10]:
import httpx

timeout = httpx.Timeout(600.0, connect=600.0)
client = httpx.Client(timeout=timeout)

In [11]:
from upstash_vector import Index

upstash_index = Index(
    url=url, 
    token=token,
    retries=5,
    retry_interval=0.2
)
upstash_index._client = client

In [14]:
vector_store = UpstashVectorStore(
    url=url, 
    token=token,
)
vector_store._index = upstash_index

In [15]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [16]:
index = VectorStoreIndex.from_documents(data, storage_context=storage_context, insert_batch_size=500)

In [17]:
index_persist = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [18]:
query_engine = index_persist.as_query_engine()

In [19]:
r = query_engine.query("What are the arguments for CharField?")
print(r.response)

max_length, db_collation
